In [1]:
#Importing the python dependencies
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from typing_extensions import TypedDict
from typing import List
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [2]:
url = "https://37d8-35-247-124-124.ngrok-free.app"

In [3]:
llm = Ollama(base_url=url, model="llama3", temperature=0.3)

## Approach 1 ##

In [4]:
def survey_question_generation(manager_role, goal, llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert survey designer.
        Your manager wants to achieve a certain goal in order to do that you've to come up with a list of atleast 10 survey questions that will
        help him achieve the goal. 
        List of questions should be in json format. Format for each json of a question should be fixed as below
        question: Survey Question
        type: Whether it is multiple_choice or open_text
        options: List of options for multiple_choice questions
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        Goal: {goal} 
        Manager Role: {manager_role} 
        Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["goal", "manager_role"],
    )
    chain = prompt | llm | JsonOutputParser()
    result = chain.invoke({"goal": goal, "manager_role": manager_role})
    return result

In [5]:
goal = "Conduct a quarterly survey of 100 financial services decision-makers to create sales content."
manager_role = "Salesforce B2B Sales Representative"
result = survey_question_generation(manager_role, goal, llm)

In [6]:
result

[{'question': 'What is your current role?',
  'type': 'multiple_choice',
  'options': ['Sales Representative',
   'Marketing Manager',
   'Financial Advisor',
   'Other (please specify)']},
 {'question': 'How often do you make purchasing decisions for financial services products?',
  'type': 'multiple_choice',
  'options': ['Daily', 'Weekly', 'Monthly', 'Quarterly', 'Rarely']},
 {'question': 'What is your primary source of information when researching financial services products?',
  'type': 'open_text'},
 {'question': 'How important are the following factors when evaluating a financial services product: (Select all that apply)',
  'type': 'multiple_choice',
  'options': ['Cost',
   'Features and Benefits',
   'Brand Reputation',
   'Customer Support',
   'Other (please specify)']},
 {'question': 'What is your preferred method of communication with sales representatives?',
  'type': 'multiple_choice',
  'options': ['Phone',
   'Email',
   'In-Person Meeting',
   'Online Chat',
   'Othe

In [7]:
## Approach 2 ##

In [8]:
class QuestionGenEngine:
    def __init__(self, llm_url):
        self.llm = Ollama(base_url=url, model="llama3", temperature=0.3)
        self.workflow = None
        self.question_gen_prompt_template = PromptTemplate(
                template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert survey designer.
                Your manager wants to achieve a certain goal in order to do that you've to come up with a list of atleast 10 survey questions that will
                help him achieve the goal. 
                List of questions should be in json format. Format for each json of a question should be fixed as below
                question: Survey Question
                type: Whether it is multiple_choice or open_text
                options: List of options for multiple_choice questions
                <|eot_id|><|start_header_id|>user<|end_header_id|>
                Goal: {goal} 
                Manager Role: {manager_role} 
                Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
                input_variables=["goal", "manager_role"],)
        self.question_gen_prompt_chain = self.question_gen_prompt_template | self.llm | JsonOutputParser()
        
        self.grader_prompt = PromptTemplate(
            template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
            your colleague has generated a relevant survey question related to a goal. 
            Give a binary score 'yes' or 'no' to indicate whether the answer is 
            useful to resolve a question. Provide the binary score as a JSON with a key 'score' and explaination with a key 'explaination'.
            <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the goal and manger role:
            \n ------- \n
            Goal: {goal}
            Manager Role: {manager_role}
            \n ------- \n
            Here is the generated survey question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
            input_variables=["goal", "manger_role", "question"],
        )
        self.grader_chain = self.grader_prompt | self.llm | JsonOutputParser()

    # Define State
    class GraphState(TypedDict):
        """
        Represents the state of our graph.

        Attributes:
            question: question
            generation: LLM generation
            documents: list of documents 
        """
        goal : str
        manager_role: str
        question : List[str]

    def generate(self, state):
        print("---GENERATE---")
        goal = state["goal"]
        manager_role = state["manager_role"]
        
        question = self.question_gen_prompt_chain.invoke({"goal": goal, "manager_role": manager_role})
        print(question)
        return {"goal": goal, "manager_role": manager_role, "question": question}

    def grade_question(self, state):
        print("---CHECK QUESTION RELEVANCE TO GOAL---")
        goal = state["goal"]
        manager_role = state["manager_role"]
        questions = state["question"]
        
        # Score each question
        filtered_question = []
        for q in questions:
            score = self.grader_chain.invoke({"goal": goal, "manager_role": manager_role, "question": q})
            grade = score['score']
            # Question relevant
            if grade.lower() == "yes":
                print("---GRADE: QUESTION RELEVANT---")
                q["explaination"] = score["explaination"]
                filtered_question.append(q)
            # Question not relevant
            else:
                print("---GRADE: QUESTION NOT RELEVANT---")
                continue
        question = filtered_question
        return {"goal": goal, "manager_role": manager_role, "question": question}

    def add_nodes(self):
        self.workflow = StateGraph(self.GraphState)

        # Define the nodes
        self.workflow.add_node("generate", self.generate) # generatae
        self.workflow.add_node("grade_question", self.grade_question) # grade question

    def build_graph(self):
        self.add_nodes()
        self.workflow.set_entry_point("generate")
        self.workflow.add_edge("generate", "grade_question")
        self.workflow.add_edge("grade_question", END)
  
        app = self.workflow.compile()
        return app

    def execute_graph(self, goal, manager_role):
        inputs = {"goal": goal, "manager_role": manager_role}
        app = self.build_graph()
        for output in app.stream(inputs):
            for key, value in output.items():
                # Node
                print(f"Node '{key}':")
                # Optional: print full state at each node
                # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
            print("\n---\n")
        return value

In [9]:
question_gen = QuestionGenEngine(url)

In [10]:
goal = "Conduct a quarterly survey of 100 financial services decision-makers to create sales content."
manager_role = "Salesforce B2B Sales Representative"

In [11]:
result = question_gen.execute_graph(goal, manager_role)

---GENERATE---
[{'question': 'What is your current role?', 'type': 'multiple_choice', 'options': ['Sales Representative', 'Financial Advisor', 'Investment Manager', 'Other (please specify)']}, {'question': 'How often do you make purchasing decisions for financial services?', 'type': 'multiple_choice', 'options': ['Daily', 'Weekly', 'Monthly', 'Quarterly', 'Rarely']}, {'question': 'What is the primary factor that influences your purchasing decisions for financial services?', 'type': 'open_text'}, {'question': 'How do you currently stay informed about new financial services and products?', 'type': 'multiple_choice', 'options': ['Industry publications', 'Social media', 'Colleagues and peers', 'Online webinars and conferences', 'Other (please specify)']}, {'question': 'What type of content would you like to see more of from financial services providers?', 'type': 'open_text'}, {'question': 'How important is it for you that a financial services provider has a strong online presence?', 'type

In [12]:
result

{'goal': 'Conduct a quarterly survey of 100 financial services decision-makers to create sales content.',
 'manager_role': 'Salesforce B2B Sales Representative',
 'question': [{'question': 'What is your current role?',
   'type': 'multiple_choice',
   'options': ['Sales Representative',
    'Financial Advisor',
    'Investment Manager',
    'Other (please specify)'],
   'explaination': "The generated survey question is relevant to the goal of creating sales content for financial services decision-makers. The question 'What is your current role?' helps identify the target audience's job function, which can inform the type of sales content that would be most effective in resonating with them. As a Salesforce B2B Sales Representative, this information is crucial in understanding the needs and pain points of the potential customers."},
  {'question': 'How often do you make purchasing decisions for financial services?',
   'type': 'multiple_choice',
   'options': ['Daily', 'Weekly', 'Monthl